# Modelling with DL and word embedding transform

In [1]:
# try weighted word embedding
# https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/

In [2]:
# https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5
# https://docs.google.com/spreadsheets/d/1hg4KKKdivoh6mG7Bena0jYdJSvzgABlWLrU0NX_vHTQ/edit#gid=49385392

In [3]:
# Using our_vocab3_w_merchant.bin

In [4]:
data_path='../../data/'
data_raw_path="../../data/raw/for third propose/"
data_man_path="../../data/manipulate/for third propose/"
model_path="../../model/modeling_third_propose/"

In [5]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
import seaborn as sns
import nltk
from string import digits
import ast
import itertools

In [6]:
stop_words_id=open(data_path+"stopwords_id.txt")
stop_words_id=list(set([i[:-1] for i in stop_words_id.readlines()]))

stop_words_en=open(data_path+"stopwords_en.txt")
stop_words_en=list(set([i[:-1] for i in stop_words_en.readlines()]))

stop_words=stop_words_id+stop_words_en

In [7]:
stop_word_add=['nambah', 'byk', 'yaaa', 'ksh', 'bs', 'kl', 'sm', 'smua',
       'seadanya', 'ajh', 'atau', 'mnta', 'klw', 'ato', 'kalo', 'abis',
       'bnyk', 'pokoknya', 'jgn', 'smuanya', 'klo', 'gitu', 'yaaaa',
       'gpp', 'disitu', 'aj', 'hehe', 'jg', 'yng', 'ya', 'habis', 'gada',
       'ajaa', 'klu', 'ush', 'terserah', 'yah', 'salah', 'brp', 'yaa',
       'aja', 'trs', 'tambahin', 'nya', 'tp', 'ajah', 'mnt', 'situ',
       'segitu', 'disatuin', 'satuin', 'gapapa', 'yg', 'gabung', 'trus',
       'biar', 'nyaa', 'klau', 'lbh']
stop_words2=stop_words+stop_word_add

# Get The Data

In [8]:
dtag=pd.read_csv(data_man_path+"data_tag_goshop3_unique.csv").drop_duplicates()
# dtag.to_csv(data_man_path+"data_tag_goshop2.csv",index=False)

In [9]:
dtext2=pd.read_csv(data_man_path+"data_gocommerce_items_sample_1_cln_w_label3.csv")
dtext2["real_tags"]=dtext2.real_tags.apply(lambda x : set(ast.literal_eval(x)))

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
dtext2["num_tags"]=dtext2.real_tags.apply(lambda x: len(x))

In [11]:
dtag["num_word"]=dtag.Keywords.apply(lambda x : len(x.split(" ")))
dtag["num_word_tag"]=dtag.Tags.apply(lambda x : len(x.split(" ")))

In [12]:
# keyword_set=set(dtag.Keywords.unique())
# keyword_set1=set([t for t in keyword_set if len(t.split(" "))==1])
# keyword_set2=set([t for t in keyword_set if len(t.split(" "))==2])
# keyword_set3=set([t for t in keyword_set if len(t.split(" "))==3])


# ts=set(dtag.Tags.unique())
# ts1=set([t for t in ts if len(t.split(" "))==1])
# ts2=set([t for t in ts if len(t.split(" "))==2])
# ts3=set([t for t in ts if len(t.split(" "))==3])

In [13]:
# dict_keyword=pd.DataFrame(dtag.Tags)
# dict_keyword.index=dtag.Keywords
# dict_keyword=dict_keyword["Tags"].to_dict()

# dict_sc=pd.DataFrame(dtag["Sub-Categories"])
# dict_sc.index=dtag.Tags
# dict_sc=dict_sc["Sub-Categories"].to_dict()

In [14]:
dtext2.sample(100).head(20)

,order_no,merchant_name,item_list,type,item_list_cln1,item_list_cln2,merchant_name_cln1,real_tags,num_tags
3041230,318321431,Waroenk K'Esa,"tempe/tahu,capcay seafood",food,"tempe tahu,capcay seafood",tempe tahu capcay seafood,waroenk k esa,"{seafood, tempe}",2
859947,473648035,-,bernardi sosis sapi 250 gr,mart,bernardi sosis sapi gr,bernardi sosis sapi gr,NaN,"{sosis, bernardi}",2
3034901,367901767,Poenya Nyonya Anina,"nasi goreng cabe ijo,cireng",food,"nasi goreng cabe ijo,cireng",nasi goreng cabe ijo cireng,poenya nyonya anina,"{nasi goreng, cabe ijo}",2
2551718,267747619,-,rujak cuka serut 1 bungkus,shop,rujak cuka serut bungkus,rujak cuka serut bungkus,NaN,{rujak},1
1997171,471438191,-,1 bolu gulung coklat tokonya Amaro depan sari...,shop,bolu gulung coklat tokonya amaro depan sari sa...,bolu gulung coklat tokonya amaro depan sari sa...,NaN,"{cokelat, bolu}",2
3204738,325519210,Amanda Brownies,brownies sarikaya pandan,food,brownies sarikaya pandan,brownies sarikaya pandan,amanda brownies,"{brownis, srikaya}",2
2056345,438477877,-,luti gendang,shop,luti gendang,luti gendang,NaN,{},0
2642212,367037258,-,sate kambing bumbu kecap 15 tusuk,shop,sate kambing bumbu kecap tusuk,sate kambing bumbu kecap tusuk,NaN,"{bumbu kecap, sate kambing}",2
1230796,199580027,-,Ayam bakar taliwang 2 pedes,shop,ayam bakar taliwang pedes,ayam bakar taliwang pedes,NaN,{ayam bakar},1
844616,426673380,-,"a t b marie susu 185 gr,khong guan malkist cra...",mart,"a t b marie susu gr,khong guan malkist cracker...",a t b marie susu gr khong guan malkist cracker...,NaN,"{susu, khong guan, malkist, kopi susu, kapal api}",5


In [15]:
len(dtext2)

3599997

### Modeling

In [16]:
import keras.backend as K

Using TensorFlow backend.


In [17]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)


def custom_loss_func2(model,X,y_true0):
    y_pred0=model.predict(X)
    return hamming_score(y_true0, y_pred0)


def print_score(y_pred, clf,y_test_tfidf):
    print("Clf: ", clf)
    print("Hamming loss: {}".format(hamming_loss(y_test_tfidf,y_pred)))
    try:
        print("Hamming score: {}".format(hamming_score(y_test_tfidf,y_pred)))
    except:
        print("Hamming score: {}".format(hamming_score(y_test_tfidf,y_pred.toarray())))
        
    print("---")    
    
def check_tag2(list_X,i):
    if i in list_X:
        return 1
    else :
        return 0
    

# https://stats.stackexchange.com/questions/207794/what-loss-function-for-multi-class-multi-label-classification-tasks-in-neural-n    
def abs_KL_div(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), None)
    y_pred = K.clip(y_pred, K.epsilon(), None)
    return K.sum( K.abs( (y_true- y_pred) * (K.log(y_true / y_pred))), axis=-1)

def hamming_loss_keras(y_true, y_pred):
    H = K.average(y_true*(1-y_pred)+(1-y_true)*y_pred)

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.metrics import hamming_loss
from sklearn.preprocessing import LabelEncoder
from skmultilearn.problem_transform import LabelPowerset,ClassifierChain
from skmultilearn.adapt import MLkNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from gensim.models import Word2Vec
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors
from collections import defaultdict
from sklearn.preprocessing import RobustScaler

In [19]:
w2v_path="../../data/w2v/"

In [20]:
# embd2=Word2Vec.load(w2v_path+'our_vocab.bin')
embd2=Word2Vec.load(w2v_path+'our_vocab3_w_merchant.bin')

In [21]:
embd2.wv.most_similar(positive="masker",topn=20)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('naturgo', 0.7355045080184937),
 ('penutup', 0.6966490745544434),
 ('hanasui', 0.6575050354003906),
 ('mask', 0.6334267854690552),
 ('girly', 0.598811686038971),
 ('topeng', 0.5978488922119141),
 ('shiseido', 0.5829647779464722),
 ('ridi', 0.5745498538017273),
 ('badda', 0.5654640793800354),
 ('neska', 0.5622670650482178),
 ('diapro', 0.5609920620918274),
 ('nexcare', 0.5497276782989502),
 ('remedi', 0.5444644093513489),
 ('mabello', 0.5415610671043396),
 ('collagen', 0.5402159094810486),
 ('slayer', 0.5255414247512817),
 ('earloop', 0.5245345830917358),
 ('lotong', 0.504840612411499),
 ('innisfree', 0.5043998956680298),
 ('namo', 0.5011873245239258)]

In [22]:
embd2.layer1_size

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).
  if __name__ == '__main__':


90

In [23]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import model_from_json

In [24]:
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, embedding.layer1_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        try:
            weight_matrix[i] = embedding[word]
        except:
            weight_matrix[i] = np.zeros(embedding.layer1_size)
    return weight_matrix

In [25]:
## remove zero tags
dtext3=dtext2[dtext2.num_tags!=0]
dtext3.head()

,order_no,merchant_name,item_list,type,item_list_cln1,item_list_cln2,merchant_name_cln1,real_tags,num_tags
0,457815035,-,fanta stroberi 390 ml,mart,fanta stroberi ml,fanta stroberi ml,NaN,{fanta},1
1,231213726,-,aqua - 1500 ml air mineral,mart,"aqua , ml air mineral",aqua ml air mineral,NaN,"{aqua, air}",2
2,413928537,-,samyang buldak cheese hot chicken ramen 140 gr...,mart,samyang buldak cheese hot chicken ramen gr ins...,samyang buldak cheese hot chicken ramen gr ins...,NaN,"{ayam, ramen, samyang, mie}",4
3,436103314,-,heineken multipack pint 330 ml x 4 pcs,mart,heineken multipack pint ml x pcs,heineken multipack pint ml x pcs,NaN,{heineken},1
4,464218735,-,"aqua - 1 5 l,yakult - 5 pcs,sari roti coklat 1...",mart,"aqua , l,yakult , pcs,sari roti coklat pcs,ene...",aqua l yakult pcs sari roti coklat pcs energen...,NaN,"{energen, sari roti, cokelat, aqua, yakult}",5


In [26]:
len(dtext3)

2997188

In [27]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(dtext3.real_tags)

MultiLabelBinarizer(classes=None, sparse_output=False)

In [28]:
joblib.dump(multilabel_binarizer,model_path+"multilabel_bin_for_DL1.pkl")

['../../model/modeling_third_propose/multilabel_bin_for_DL1.pkl']

In [29]:
len(multilabel_binarizer.classes_.tolist())

1129

In [30]:
all_tag_used=multilabel_binarizer.classes_.tolist()
# keyword_set=set(dtag.Keywords.unique())
# keyword_set1=set([t for t in keyword_set if len(t.split(" "))==1])
# keyword_set2=set([t for t in keyword_set if len(t.split(" "))==2])
# keyword_set3=set([t for t in keyword_set if len(t.split(" "))==3])


ts=set(all_tag_used)
ts1=set([t for t in ts if len(t.split(" "))==1])
ts2=set([t for t in ts if len(t.split(" "))==2])
ts3=set([t for t in ts if len(t.split(" "))==3])

In [31]:
train2, test2 = train_test_split(dtext3.sample(2850000,random_state=231), random_state=42, test_size=0.2, shuffle=True)

X_train2=train2.item_list_cln2
X_test2=test2.item_list_cln2

Y_train2=multilabel_binarizer.transform(train2.real_tags)
Y_test2=multilabel_binarizer.transform(test2.real_tags)

In [32]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_train2)

In [33]:
joblib.dump(tokenizer,model_path+"tokenizer_for_DL1.pkl")

['../../model/modeling_third_propose/tokenizer_for_DL1.pkl']

In [34]:
## Training
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(X_train2)
# pad sequences
# max_length = max([len(s.split()) for s in X_train2]) #340
max_length = 50
Xtr2 = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [35]:
max_length

50

In [36]:
len_word=np.array([len(s.split()) for s in X_train2])

In [37]:
Xtr2.shape

(2280000, 50)

In [38]:
np.percentile(len_word,99)

42.0

In [39]:
## Testing
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(X_test2)
# pad sequences
# max_length = max([len(s.split()) for s in X_test2])
Xte2 = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [40]:
vocab_size = len(tokenizer.word_index) + 1

In [41]:
embedding_vectors = get_weight_matrix(embd2, tokenizer.word_index)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:11: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).


In [42]:
embedding_layer = Embedding(vocab_size, embd2.layer1_size, weights=[embedding_vectors], 
                            input_length=max_length, trainable=False)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).
  if __name__ == '__main__':


In [43]:
Y_train2.shape[1]

1129

In [44]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=65, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(600, activation='relu'))
model.add(Dense(600, activation='relu'))
model.add(Dense(Y_train2.shape[1], activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 90)            16557120  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 65)            29315     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 65)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 975)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 600)               585600    
_________________________________________________________________
dense_2 (Dense)              (None, 600)               360600    
_________________________________________________________________
dense_3 (Dense)              (None, 1129)              678529    
Total para

In [45]:
# fit network
model.fit(Xtr2, Y_train2, epochs=2, batch_size=1500, verbose=1)

Epoch 1/2
2280000/2280000 [==============================] - 1633s 716us/step - loss: 0.0149 - acc: 0.9979
Epoch 2/2
2280000/2280000 [==============================] - 2026s 888us/step - loss: 0.0024 - acc: 0.9993


In [46]:
model

In [47]:

# evaluate
loss, acc = model.evaluate(Xte2, Y_test2, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 99.940167


In [48]:
# serialize model to JSON
model_json = model.to_json()
with open(model_path+"wv_DL.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(model_path+"wv_DL.h5")
print("Saved model to disk")

Saved model to disk


In [44]:
# # later...
 
# load json and create model
json_file = open(model_path+"wv_DL.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
go_shop_model = model_from_json(loaded_model_json)
# load weights into new model
go_shop_model.load_weights(model_path+"wv_DL.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
go_shop_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# score = go_shop_model.evaluate(Xte2, Y_test2, verbose=0)
# print("%s: %.2f%%" % (go_shop_model.metrics_names[1], score[1]*100))

Loaded model from disk


In [45]:
## DL1
y_pred = go_shop_model.predict(Xte2)

In [58]:
def transform_y(yp,a=0.22):
    YY=(yp>a).astype(int)
    while YY.sum()==0 and a>=0.34:
        a=a-0.02
        YY=(yp>a).astype(int)
    return YY

In [59]:
# aa=[0.1105+((0.012)*i) for i in range(60)]
# for a in aa:
#     y_pred0=np.array([transform_y(yp,a) for yp in y_pred])
#     hs=hamming_loss(Y_test2, y_pred0)
#     print(a,hs)

In [60]:
# y_pred2=np.array([(yp>0.18).astype(int) for yp in y_pred])
y_pred2=np.array([transform_y(yp,a=0.53) for yp in y_pred])
test2["tag_pred"]=multilabel_binarizer.inverse_transform(y_pred2)

In [61]:
hamming_loss(Y_test2,y_pred2)

0.0005957453420974935

In [62]:
go_shop_model

In [69]:
test2.sample(300).head(10)

,order_no,merchant_name,item_list,type,item_list_cln1,item_list_cln2,merchant_name_cln1,real_tags,num_tags,tag_pred
1261743,379318035,-,Nasi sayur 2 bgks pake ayam rendang dada ya,shop,nasi sayur bgks pake ayam rendang dada ya,nasi sayur bgks pake ayam rendang dada ya,NaN,"{nasi sayur, rendang, ayam}",3,"(ayam, nasi sayur, rendang)"
2662413,336076806,-,BAKSO BRI,shop,bakso bri,bakso bri,NaN,{bakso},1,"(bakso,)"
3438290,GF-455228846,Martabak Bandung Zaenal,"martabak biasa keju,martabak super coklat",food,"martabak biasa keju,martabak super coklat",martabak biasa keju martabak super coklat,martabak bandung zaenal,"{cokelat, martabak}",2,"(cokelat, martabak)"
1483692,424463387,-,"pesan lomie 2 porsi yg di depan alfamart,, min...",shop,"pesan lomie porsi yg di depan alfamart, minta ...",pesan lomie porsi yg di depan alfamart minta j...,NaN,"{lomie, jeruk, alfamart}",3,"(alfamart, jeruk)"
1599209,475134764,-,"Rujak Cingur Joko Dolog: 1 paling pedas, 1 sed...",shop,"rujak cingur joko dolog, paling pedas, sedang,...",rujak cingur joko dolog paling pedas sedang ke...,NaN,"{rujak, kerupuk, teh}",3,"(kerupuk, rujak, teh)"
2800843,309108854,Pempek Yukyan,pempek kulit,food,pempek kulit,pempek kulit,pempek yukyan,{pempek},1,"(pempek,)"
2558575,184670443,-,15 box sate. (Tlg jgn lupa mintakan kwitansi y...,shop,"box sate, tlg jgn lupa mintakan kwitansi yg su...",box sate tlg jgn lupa mintakan kwitansi yg sud...,NaN,{sate},1,"(sate,)"
2975005,323319914,J.CO Donuts & Coffee,donut 1/2 lusin,food,donut lusin,donut lusin,j co donuts coffee,{donut},1,"(donut,)"
640134,293555960,-,"monde bourbon chocolate 150gr,monde genji pi...",mart,"monde bourbon chocolate gr,monde genji pie gr,...",monde bourbon chocolate gr monde genji pie gr ...,NaN,"{indomie, ayam, susu, monde, sari roti, minyak...",15,"(blue band, cokelat, monde, roti tawar, sari r..."
1200781,406153414,-,nasi uduk emak jutek. sebelah musolah. pake go...,shop,"nasi uduk emak jutek, sebelah musolah, pake go...",nasi uduk emak jutek sebelah musolah pake gore...,NaN,"{uduk, gorengan, tempe}",3,"(gorengan, tempe, uduk)"


In [70]:
test1=test2[["item_list_cln2","real_tags","tag_pred"]]

In [71]:
# for c in ts:
#     cek_tag=test1.real_tags.apply(lambda x : check_tag2(x,c))
#     test1[c]=cek_tag

In [72]:
acc=[]
benc=[]
lrate=[]
auc_list=[]
for category in list(ts):
    # compute the testing accuracy
    True_tag=test2.real_tags.apply(lambda x: check_tag2(x,category)).as_matrix()
    Pred_tag=test2.tag_pred.apply(lambda x: check_tag2(x,category)).as_matrix()
    
    acc_score=accuracy_score(True_tag,Pred_tag)
    try:
        auc_cat=roc_auc_score(True_tag,Pred_tag)
    except:
        auc_cat=0.001
    
    print('Test accuracy is {}'.format(acc_score))
    if True_tag.mean()<0.5:
#         benchm=1-test1[category].mean()
        benchm=1-True_tag.mean()
    else:
#         benchm=test1[category].mean()
        benchm=True_tag.mean()
    print('diff with benchmark {}'.format(acc_score-benchm))
    
    acc.append(acc_score)
    benc.append(benchm)
    lrate.append(((acc_score-benchm)/float(benchm))*100)
    auc_list.append(auc_cat)
    
dFRes2=pd.DataFrame({"tags":list(ts),"accuracy":acc,"benchmark":benc,"AUC":auc_list,"learn_rate(%)":lrate})

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Test accuracy is 0.9998684210526316
diff with benchmark 1.4035087719355133e-05


/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Test accuracy is 0.9999543859649123
diff with benchmark 8.421052631579773e-05


/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


...

Test accuracy is 0.9994684210526316
diff with benchmark 0.00012631578947364108


/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Test accuracy is 0.9989350877192983
diff with benchmark 0.00843508771929824


/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Test accuracy is 0.9998824561403509
diff with benchmark 0.00021578947368416923


In [73]:
dFRes2["learn_rate(%)"].describe()

count    1129.000000
mean        0.154335
std         0.482264
min        -0.019486
25%         0.009301
50%         0.035459
75%         0.124549
max         9.131472
Name: learn_rate(%), dtype: float64

In [74]:
dFRes2["accuracy"].describe()

count    1129.000000
mean        0.999404
std         0.000916
min         0.984354
25%         0.999316
50%         0.999665
75%         0.999833
max         1.000000
Name: accuracy, dtype: float64

In [75]:
dFRes2["AUC"].describe()

count    1129.000000
mean        0.780872
std         0.140805
min         0.001000
25%         0.683931
50%         0.806104
75%         0.894830
max         1.000000
Name: AUC, dtype: float64

In [76]:
dFRes2.sample(100).head(15)

,tags,accuracy,benchmark,AUC,learn_rate(%)
783,opor,0.999525,0.999256,0.699278,0.026862
153,softlens,0.999647,0.996777,0.967538,0.287946
794,vitacimin,0.999719,0.999633,0.648314,0.008600
826,ayam kampung,0.998391,0.996330,0.873224,0.206900
858,donut,0.999388,0.994981,0.955005,0.442925
738,emping,0.999261,0.998953,0.659117,0.030910
625,natur,0.999726,0.999505,0.739354,0.022116
130,ayam,0.984354,0.901989,0.950578,9.131472
263,ikan bakar,0.999712,0.999005,0.869474,0.070772
933,adidas,0.999875,0.999793,0.716098,0.008247


In [77]:
# ', '.join(dFRes2[dFRes2.AUC<=0.8].tags.tolist())

In [78]:
len(ts)

1129

In [79]:
dFRes2[(dFRes2.AUC>0.8)&(dFRes2["learn_rate(%)"]>0)][["accuracy","AUC"]].describe()

,accuracy,AUC
count,584.000000,584.000000
mean,0.999204,0.890599
std,0.001192,0.051824
min,0.984354,0.800054
25%,0.999076,0.846442
50%,0.999583,0.891572
75%,0.999797,0.935695
max,1.000000,1.000000


In [80]:
good_tags=', '.join(dFRes2[(dFRes2.AUC>0.8)&(dFRes2["learn_rate(%)"]>0)].tags.tolist())
ss=open(model_path+"good_tags_DL1.txt",'w')
ss.write(good_tags)
ss.close()